In [ ]:
http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/demo_r_pjanind3?indic_de=PC_Y25_44&precision=2&unit=PC

In [4]:
from census import Census
from us import states
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [5]:
census_api_k = os.getenv('census_api_key')

In [70]:
c = Census(census_api_k, year=2018)

In [72]:
# c.acs5dp.zipcode(median_age, 781)

[]

In [61]:
median_age = 'DP05_0018E'
range_25_34 = 'DP05_0010PE'
range_35_44 = 'DP05_0011PE'

school_enroll = 'DP02_0052E'
school_enrollP = 'DP02_0052PE'

preschool_enroll = 'DP02_0053E'
preschool_enrollP = 'DP02_0053PE'

kinderg_enroll = 'DP02_0054E'
kinderg_enrollP = 'DP02_0054PE'

elem_enroll = 'DP02_0055E'
elem_enrollP = 'DP02_0055PE'

high_enroll = 'DP02_0056E'
high_enrollP = 'DP02_0056PE'

hschool_gradP = 'DP02_0066PE'
bach_gradP = 'DP02_0067PE'